In [9]:
import glob
import os
from pathlib import Path

In [ ]:
def parse_RPT(rpt_file):
    

In [7]:
if not os.path.isdir("raw_reports"):
    raise Exception("Folder 'raw_reports' not found. Consider creating it and moving RPT files there.")

Exception: Folder 'raw_reports' not found. Consider creating it and moving RPT files there.

In [10]:
Path("./parsed_reports").mkdir(parents=True, exist_ok=True)

In [15]:
for rpt_file in glob.iglob("./raw_reports/*.RPT"):
    header, data = parse_RPT(rpt_file)

NameError: name 'parse_RPT' is not defined